In [394]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [395]:
pwd

'C:\\Users\\Justi\\Bootcamp Course Work\\Github\\Job_salary_prediction_2'

In [396]:
file_path = "Data/Train_rev1.csv"
df = pd.read_csv(file_path)
df.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


In [397]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  244768 non-null  int64 
 1   Title               244767 non-null  object
 2   FullDescription     244768 non-null  object
 3   LocationRaw         244768 non-null  object
 4   LocationNormalized  244768 non-null  object
 5   ContractType        65442 non-null   object
 6   ContractTime        180863 non-null  object
 7   Company             212338 non-null  object
 8   Category            244768 non-null  object
 9   SalaryRaw           244768 non-null  object
 10  SalaryNormalized    244768 non-null  int64 
 11  SourceName          244767 non-null  object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


In [398]:
# based on this those 3 features should be dropped before removing the null values
df.isnull().sum()

Id                         0
Title                      1
FullDescription            0
LocationRaw                0
LocationNormalized         0
ContractType          179326
ContractTime           63905
Company                32430
Category                   0
SalaryRaw                  0
SalaryNormalized           0
SourceName                 1
dtype: int64

In [399]:
# Drop the columns that we won't need, Company, ContractType, ContractTime, SalaryRaw, LocationRaw, ID
new_df = df.drop(columns = ["Company", "ContractType", "ContractTime", "SalaryRaw", "LocationRaw", "Id", "SourceName"])
new_df.head()

,Title,FullDescription,LocationNormalized,Category,SalaryNormalized
0,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,Dorking,Engineering Jobs,25000
1,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,Glasgow,Engineering Jobs,30000
2,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,Hampshire,Engineering Jobs,30000
3,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,Surrey,Engineering Jobs,27500
4,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...",Surrey,Engineering Jobs,25000


In [400]:
len(new_df)

244768

In [401]:
# check datatypes again
new_df.dtypes

Title                 object
FullDescription       object
LocationNormalized    object
Category              object
SalaryNormalized       int64
dtype: object

In [402]:
# find how many null values are in the new_df
new_df.isnull().sum()

Title                 1
FullDescription       0
LocationNormalized    0
Category              0
SalaryNormalized      0
dtype: int64

In [403]:
# drop the null values of this dataframe
new_df.dropna(inplace = True)

In [404]:
new_df.isnull().sum()

Title                 0
FullDescription       0
LocationNormalized    0
Category              0
SalaryNormalized      0
dtype: int64

In [405]:
len(new_df)

244767

In [406]:
sliced_df = new_df.loc[:10000, :]

In [407]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Code to download corpora
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [408]:
lemmatizer = WordNetLemmatizer()

In [409]:
stemmer = SnowballStemmer("english")

In [410]:
def clean_text(article):
    sw = set(stopwords.words('english'))
    sw_addons = {"k", "uk","also"} 
    # Substitute everything that is not a letter with an empty string
    regex = re.compile("[^a-zA-Z ]")
    # we sub in an extra character for anything that is not a character from the
    # above line of code
    re_clean = regex.sub('', article)
    # tokenize each word in the sentence
    words = word_tokenize(re_clean)
    # obtain the root word for each word 
    #stem = [stemmer.stem(word) for word in words]
    lem = [lemmatizer.lemmatize(word) for word in words]
    # obtain an output that is all lowercase and not in the stop words
    #output = [word.lower() for word in stem if word.lower() not in sw.union(sw_addons)]
    output = [word.lower() for word in lem if word.lower() not in sw.union(sw_addons)]
    output = ' '.join(output)
    return output

In [411]:
# test function on sliced df to make sure it is correct
clean_text(sliced_df["FullDescription"][0])

'engineering systems analyst dorking surrey salary client located dorking surrey looking engineering systems analyst client provides specialist software development keywords mathematical modelling risk analysis system modelling optimisation miser pioneeer engineering systems analyst dorking surrey salary'

In [412]:
# create new column that has the clean description of the job
sliced_df['CleanDescription'] = sliced_df['FullDescription'].apply(clean_text)
sliced_df.head()

C:\Users\Justi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Title,FullDescription,LocationNormalized,Category,SalaryNormalized,CleanDescription
0,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,Dorking,Engineering Jobs,25000,engineering systems analyst dorking surrey sal...
1,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,Glasgow,Engineering Jobs,30000,stress engineer glasgow salary currently looki...
2,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,Hampshire,Engineering Jobs,30000,mathematical modeller simulation analyst opera...
3,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,Surrey,Engineering Jobs,27500,engineering systems analyst mathematical model...
4,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...",Surrey,Engineering Jobs,25000,pioneer miser engineering systems analyst dork...


In [413]:
# Now we want to get the TD-IDF of the CleanDescription Column

# create varialbe to store tfidf vectorizer
vectorizer_2 = TfidfVectorizer(stop_words="english", min_df = .30)
tf_idf_vectorizer = vectorizer_2.fit_transform(sliced_df["CleanDescription"])
tf_idf_df = pd.DataFrame(tf_idf_vectorizer.toarray(), columns=vectorizer_2.get_feature_names())
tf_idf_df.head()

,apply,based,business,candidate,client,company,cv,excellent,experience,job,...,required,role,salary,service,skill,successful,support,team,work,working
0,0.0,0.0,0.0,0.000000,0.594892,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.733651,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.375543,0.0,...,0.149976,0.226400,0.301628,0.0,0.266005,0.000000,0.162100,0.354394,0.110231,0.229294
2,0.0,0.0,0.0,0.332583,0.585311,0.0,0.0,0.112898,0.299575,0.0,...,0.119638,0.090301,0.000000,0.0,0.318293,0.371293,0.129309,0.000000,0.175865,0.274366
3,0.0,0.0,0.0,0.000000,0.416074,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.513124,0.0,0.000000,0.527874,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.000000,0.375726,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.926731,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [415]:
# Calculating the COUNT for the working corpus.
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english",min_df=.30)
count_vectorizer = vectorizer.fit_transform(sliced_df["CleanDescription"])
words_df = pd.DataFrame(count_vectorizer.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,apply,based,business,candidate,client,company,cv,excellent,experience,job,...,required,role,salary,service,skill,successful,support,team,work,working
0,0,0,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,4,0,...,1,2,2,0,2,0,1,3,1,2
2,0,0,0,3,6,0,0,1,4,0,...,1,1,0,0,3,3,1,0,2,3
3,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


In [416]:
filtered_df = words_df.replace([2,3,4,5,6,7,8,9],1)

In [417]:
# Now set the vectorizer on the CleanDescription column and store it in a new variable
#tf_idf_2 = vectorizer.fit_transform(sliced_df['CleanDescription'])
# we now want to set the tf_idf to a dataframe
#tf_score_df_2 = pd.DataFrame(tf_idf_2.toarray(), columns = vectorizer.get_feature_names())
#tf_score_df_2.head()

In [418]:
combined_df = pd.concat([sliced_df, filtered_df], axis = 1)
combined_df.head()

,Title,FullDescription,LocationNormalized,Category,SalaryNormalized,CleanDescription,apply,based,business,candidate,...,required,role,salary,service,skill,successful,support,team,work,working
0,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,Dorking,Engineering Jobs,25000.0,engineering systems analyst dorking surrey sal...,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,Glasgow,Engineering Jobs,30000.0,stress engineer glasgow salary currently looki...,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
2,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,Hampshire,Engineering Jobs,30000.0,mathematical modeller simulation analyst opera...,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
3,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,Surrey,Engineering Jobs,27500.0,engineering systems analyst mathematical model...,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...",Surrey,Engineering Jobs,25000.0,pioneer miser engineering systems analyst dork...,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [419]:
combined_df.isnull().sum()

Title                 1
FullDescription       1
LocationNormalized    1
Category              1
SalaryNormalized      1
CleanDescription      1
apply                 1
based                 1
business              1
candidate             1
client                1
company               1
cv                    1
excellent             1
experience            1
job                   1
join                  1
level                 1
looking               1
management            1
need                  1
opportunity           1
recruitment           1
required              1
role                  1
salary                1
service               1
skill                 1
successful            1
support               1
team                  1
work                  1
working               1
dtype: int64

In [420]:
# drop the 2 description columns as we no longer need them
combined_df = combined_df.drop(columns = ["FullDescription", "CleanDescription"])
combined_df.head()

,Title,LocationNormalized,Category,SalaryNormalized,apply,based,business,candidate,client,company,...,required,role,salary,service,skill,successful,support,team,work,working
0,Engineering Systems Analyst,Dorking,Engineering Jobs,25000.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stress Engineer Glasgow,Glasgow,Engineering Jobs,30000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
2,Modelling and simulation analyst,Hampshire,Engineering Jobs,30000.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
3,Engineering Systems Analyst / Mathematical Mod...,Surrey,Engineering Jobs,27500.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"Pioneer, Miser Engineering Systems Analyst",Surrey,Engineering Jobs,25000.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [421]:
# use get dummies to turn the category columns into number columns
encoded_df = pd.get_dummies(combined_df)
encoded_df.head()

,SalaryNormalized,apply,based,business,candidate,client,company,cv,excellent,experience,...,Category_Other/General Jobs,"Category_PR, Advertising & Marketing Jobs",Category_Property Jobs,Category_Retail Jobs,Category_Sales Jobs,Category_Scientific & QA Jobs,Category_Social work Jobs,Category_Teaching Jobs,Category_Trade & Construction Jobs,Category_Travel Jobs
0,25000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,30000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,30000.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,27500.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,25000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [422]:
len(encoded_df)

10001

In [423]:
# split the dataset into X and y
X = encoded_df.drop(columns = ["SalaryNormalized"])
y = encoded_df["SalaryNormalized"].values.reshape(-1,1)

In [424]:
# look at the shape of each data set
X.shape

(10001, 9228)

In [425]:
y.shape

(10001, 1)

In [426]:
# lets import train test split to split the data up
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=78)

In [427]:
# import the scaler
scaler = StandardScaler()

In [428]:
# scale the training data
scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [429]:
# import PCA for demionality reduction, we have alot of columns so lets condense them down
from sklearn.decomposition import PCA

In [430]:
# set the parameters for the number of components
pca = PCA(n_components=4)
# fit_transform X_train and X_test to pca
X_pca_train = pca.fit_transform(X_train_scaled)
X_pca_test = pca.fit_transform(X_test_scaled)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# import linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_pca_train, y_train)

In [ ]:
print(f'Train score: {model.score(X_pca_train, y_train)}')
print(f"Test score: {model.score(X_pca_test, y_test)}")

In [ ]:
# predict the X_test values
predictions_x_test = model.predict(X_pca_test)


In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_model = Ridge()
ridge_model.fit(X_pca_train, y_train)

In [ ]:
print(f'Train score: {ridge_model.score(X_pca_train, y_train)}')
print(f"Test score: {ridge_model.score(X_pca_test, y_test)}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(n_estimators= 100, min_samples_split = 4, min_samples_leaf=2, random_state = 78)

In [ ]:
# fit the random forest regressor to the x_train and y _train
rfr.fit(X_pca_train, y_train)

In [ ]:
print(f'Train score: {rfr.score(X_pca_train, y_train)}')
print(f"Test score: {rfr.score(X_pca_test, y_test)}")

## Try using TF-IDF

In [ ]:
# try using tf_idf instead of the count
# create varialbe to store tfidf vectorizer
vectorizer_2 = TfidfVectorizer(stop_words="english", max_df = .30)
tf_idf_vectorizer = vectorizer_2.fit_transform(sliced_df["CleanDescription"])
tf_idf_df = pd.DataFrame(tf_idf_vectorizer.toarray(), columns=vectorizer_2.get_feature_names())
tf_idf_df.head()

In [ ]:
combined_df_2 = pd.concat([sliced_df, tf_idf_df], axis = 1)
combined_df_2.head()

In [ ]:
combined_df_2 = pd.get_dummies(combined_df_2)
combined_df_2.shape

In [ ]:
X_2 = combined_df_2.drop(columns = ["SalaryNormalized"])
y_2 = combined_df_2["SalaryNormalized"]

In [ ]:
# lets import train test split to split the data up
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2,
                                                   y_2,
                                                   random_state=78)

In [ ]:
scaler_2 = scaler.fit(X_train_2)
X_train_scaled_2 = scaler.transform(X_train_2)
X_test_scaled_2 = scaler.transform(X_test_2)

In [ ]:
X_pca_train_2 = pca.fit_transform(X_train_scaled_2)
X_pca_test_2 = pca.fit_transform(X_test_scaled_2)

In [ ]:
rfr_2 = RandomForestRegressor(n_estimators= 1000, min_samples_split = 3, min_samples_leaf=1, random_state = 78)

In [ ]:
# fit the random forest regressor to the x_train and y _train
rfr_2.fit(X_pca_train_2, y_train_2)

In [ ]:
print(f'Train score: {rfr_2.score(X_pca_train_2, y_train_2)}')
print(f"Test score: {rfr_2.score(X_pca_test_2, y_test_2)}")